In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('brand_data/brand_data_count.csv').iloc[:, 1:]

In [8]:
df.iloc[3::4]

,brand,name
3,羅技 Logitech,611
7,蘋果 Apple,489
11,KIRIN,433
15,Za,373
19,黑人牙膏,328
...,...,...
503,比菲多,31
507,Play Station,31
511,Jordan,30
515,Mdmmd 明洞國際,30


In [15]:
data = pd.read_csv('brand_data/brand_final.csv')

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
data[data['bra_整合'] == '羅技logitech']

,name_is_onlynum,name,brand,brand_en,brand_zh,new_brand,bra_整合,new_brand = bra,Unnamed: 8
14296,False,羅技g604lightspeed無線電競滑鼠,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14297,False,羅技m350滑鼠風格上蓋-粉櫻初綻,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14298,False,110260241279羅技珍珠白/k580超薄跨平台藍牙鍵盤,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14299,False,羅技g512機械電競鍵盤gx線性紅軸,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14300,False,羅技m350鵝卵石無線滑鼠玫瑰粉-m350rose,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
...,...,...,...,...,...,...,...,...,...
14902,False,6-moulg.m280黑2.4g,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14903,False,羅技m331舒適靜音滑鼠-黑型號:910-004917,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14904,False,logitech羅技mxanywhere3無線精巧藍牙高效滑鼠anywhere3,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
14905,False,logitech羅技m590多工靜音無線滑鼠flow技術黑紅灰24h到貨【官方旗艦店】,羅技 logitech,logitech,羅技,羅技logitech,羅技logitech,O,NaN
